In [1]:
import requests
import csv
import pandas as pd
import re

In [2]:
with open ("Entrees_BSPE-1921.tsv") as BSPE:
    df = pd.read_csv(BSPE,sep='\t')

In [3]:
df

,Unnamed: 0,article_id,article_auteur,article_titre,article_page,revue_annee,revue_numero
0,0,BSPE-1-1,LAVERAN (Alphonse),Discours d’ouverture.,1,1908,1
1,1,BSPE-1-15,"MARTIN (Louis), DARRÉ",Sur les symptômes nerveux du début de la malad...,15,1908,1
2,2,BSPE-1-18,SERGENT (ED.),La fièvre méditerranéenne en Algérie. Note pré...,18,1908,1
3,3,BSPE-1-22,REMY (F. J.),Le Debab dans la région de Barika.,22,1908,1
4,4,BSPE-1-25,"SERGENT(ED.),SERGENT(ÉT.), LEDOUX (ED.)",Le Debab dans la région de la Zousfana (SudOra...,25,1908,1
...,...,...,...,...,...,...,...
2438,2438,BSPE-14-662,ROUBAUD (E.),Le bien-être dans le Paludisme et les maladies...,662,1921,14
2439,2439,BSPE-14-666,JOUVEAU-DUBREUIL (H.),Le Béribéri au Setchouan,666,1921,14
2440,2440,BSPE-14-685,"DONATIEN (A.), SERGENT (ED., ET.), PARROT (L.)",Le Paludisme en Corse. Etude épidémiologique.,685,1921,14
2441,2441,BSPE-14-710,LAVIER (G.),Flagellés parasites intestinaux du campagnol i...,710,1921,14


In [4]:
df.describe(include=object)

,article_id,article_auteur,article_titre
count,2443,2443,2443
unique,2375,924,2408
top,BSPE-14-633,LAVERAN (Alphonse),Allocution du Président
freq,4,103,4


In [5]:
df["article_auteur"].value_counts()


LAVERAN (Alphonse)                                            103
BRUMPT (Emile)                                                 38
LÉGER (Marcel)                                                 38
MARCHOUX (Emile)                                               32
ROUBAUD (É.)                                                   31
                                                             ... 
RODHAIN (J.), PONS (C.), VANDENBRANDEN (F.), BEQUAERT (J.)      1
SOULIÉ (H.)                                                     1
CONSEIL (E.), TRIBAUDEAU (J.)                                   1
DE BERGEVIN (E.), SERGENT (ÉT.)                                 1
WYDOOGHE (J.)                                                   1
Name: article_auteur, Length: 924, dtype: int64

### Gérer un problème avec l'annotation des frères Sergent : 

In [6]:
EdSergent = r"SERGENT(\s)?\(ED(M)?.\)"
EtSergent= r"SERGENT(\s)?\((E|É)T\.\)"
EtSergentSpé = r"\, \(ÉT\.\)"
EtEdSergent = r"SERGENT \(ED\.\, ET\.\)"
EtEdSergent2 = r"SERGENT \(ETIENNE\(\)\, EDMOND\)"

for i in range (len(df)):
    auteur1 = df.iloc[i,2]
    if re.search(EdSergent,auteur1):
        df.iloc[i,2]= re.sub(EdSergent,r"SERGENT (Edmond)", df.iloc[i,2])

    if re.search(EtSergent,auteur1):
        df.iloc[i,2] = re.sub(EtSergent,r"SERGENT (Etienne)", df.iloc[i,2])    

    if re.search(EtSergentSpé,auteur1):
        df.iloc[i,2] = re.sub(EtSergentSpé,r"\, SERGENT (Etienne)", df.iloc[i,2])    

    if re.search(EtEdSergent,auteur1):
        df.iloc[i,2] = re.sub(EtEdSergent,r"SERGENT (Etienne), SERGENT (Edmond)", df.iloc[i,2])    

    if re.search(EtEdSergent2,auteur1):
        df.iloc[i,2] = re.sub(EtEdSergent2,r"SERGENT (Etienne), SERGENT (Edmond)", df.iloc[i,2])    

##### Vérification : 

# for i in range (len(df)):
#     if re.search(r"SERGENT",df.iloc[i,2]):
#         print(df.iloc[i,1],"----",df.iloc[i,2])

Attention, il me manque toujours un problème à aller chercher à la main : "SERGENT (E.)" au BSPE-6-579 

### Séparer les auteurs d'un même article.

In [7]:
## Création de nouvelles colonnes pour accueilleur les auteurs 
df["article_auteur2"] = df.apply(lambda _: '', axis=1)
df["article_auteur3"] = df.apply(lambda _: '', axis=1)
df["article_auteur4"] = df.apply(lambda _: '', axis=1)

In [8]:

    
for i in range (len(df)):
    auteur1 = df.iloc[i,2]
    #Remplir la colonne des "auteurs2"
    if re.search(r'\,(.*)',auteur1):
        df.iloc[i,2] = re.search(r'(.*)\,(.*)',auteur1).group(1)
        df.iloc[i,7] = re.search(r'(.*)\,(.*)',auteur1).group(2)

for i in range (len(df)):
    auteur1 = df.iloc[i,2]
    #Remplir la colonne des "auteurs3"
    if re.search(r'\,(.*)',auteur1):
        df.iloc[i,2] = re.search(r'(.*)\,(.*)',auteur1).group(1)
        df.iloc[i,8] = re.search(r'(.*)\,(.*)',auteur1).group(2)
        
for i in range (len(df)):
    auteur1 = df.iloc[i,2]
    #Remplir la colonne des "auteurs4"
    if re.search(r'\,(.*)',auteur1):
        df.iloc[i,2] = re.search(r'(.*)\,(.*)',auteur1).group(1)
        df.iloc[i,9] = re.search(r'(.*)\,(.*)',auteur1).group(2)

In [9]:
df

,Unnamed: 0,article_id,article_auteur,article_titre,article_page,revue_annee,revue_numero,article_auteur2,article_auteur3,article_auteur4
0,0,BSPE-1-1,LAVERAN (Alphonse),Discours d’ouverture.,1,1908,1,,,
1,1,BSPE-1-15,MARTIN (Louis),Sur les symptômes nerveux du début de la malad...,15,1908,1,DARRÉ,,
2,2,BSPE-1-18,SERGENT (Edmond),La fièvre méditerranéenne en Algérie. Note pré...,18,1908,1,,,
3,3,BSPE-1-22,REMY (F. J.),Le Debab dans la région de Barika.,22,1908,1,,,
4,4,BSPE-1-25,SERGENT (Edmond),Le Debab dans la région de la Zousfana (SudOra...,25,1908,1,LEDOUX (ED.),SERGENT (Etienne),
...,...,...,...,...,...,...,...,...,...,...
2438,2438,BSPE-14-662,ROUBAUD (E.),Le bien-être dans le Paludisme et les maladies...,662,1921,14,,,
2439,2439,BSPE-14-666,JOUVEAU-DUBREUIL (H.),Le Béribéri au Setchouan,666,1921,14,,,
2440,2440,BSPE-14-685,DONATIEN (A.),Le Paludisme en Corse. Etude épidémiologique.,685,1921,14,PARROT (L.),SERGENT (Edmond),SERGENT (Etienne)
2441,2441,BSPE-14-710,LAVIER (G.),Flagellés parasites intestinaux du campagnol i...,710,1921,14,,,


In [10]:
#Le maximum de contributions à un même article est de 8 !!!!

for i in range (len(df)):
    #Remplir la colonne des "auteurs4"
    if re.search(r'\,(.*)',df.iloc[i,2]):
        print(df.iloc[i,2])

BORREL (A.), VINCENT (H.)
GOUZIEN (P.), NATTAN-LARRIER (L.)
BERTRAND (Louis-Edmond), BILLET (A.), GRALL (CH.), ROBERT
NOC (F.), GRALL (CH.)
BROQUET (CH.), JEANSELME (E.), BERTRAND (Louis-Edmond)
PRIMET (E.), BRÉAUDAT, LE DANTEC (A.)
SERGENT (Edmond),SERGENT (Etienne)
JEANSELME (E.), MARTIN (Louis)
YAKIMOFF (W.L.), SCHOKHOR (N.I.)
YAKIMOFF (W.-L.), SCHOKHOR (N.-J.), WENIAMINOFF (S.-D.)
YAKIMOFF (W.L.), SCHOKHOR (N.J.), KOSELKINE (P.-M.), WENIAMINOFF (S.-D.)
VAILLARD, GRALL (CH.)
NICOLLE (CHARLES), CUENOD (A.)


### Séparer le prénom du nom de l'auteur

In [14]:
#Regex NOM (Prénom)
NomPrenom = r'([A-Z]) (\(.*\))'

#Nouvelle colonne pour le prénom auteur 1 => Située à la colonne n°10
df["article_auteur1_prenom"] = df.apply(lambda _: '', axis=1)
#Nouvelle colonne pour le prénom auteur 2 => Située à la colonne n°11
df["article_auteur2_prenom"] = df.apply(lambda _: '', axis=1)
#Nouvelle colonne pour le prénom auteur 3 => Située à la colonne n°12
df["article_auteur_prenom"] = df.apply(lambda _: '', axis=1)
#Nouvelle colonne pour le prénom auteur 4 => Située à la colonne n°13
df["article_auteur4_prenom"] = df.apply(lambda _: '', axis=1)

#Dictionnaire faisant correspondre de l'ensemble des colonnes contenant NOM Prénom de l'auteur :

dico_col_auteursPrenom_nom = {2:10,7:11,8:12,9:13}

for nom_auteur in dico_col_auteursPrenom_nom :
    for i in range (len(df)):
    #     print(df.iloc[i,0],type(df.iloc[i,2]))
        if df.iloc[i,nom_auteur] is not None and df.iloc[i,nom_auteur] != "":
#             print(df.iloc[i,0],df.iloc[i,key])
            if re.search(NomPrenom,df.iloc[i,nom_auteur]) :
                df.iloc[i,dico_col_auteursPrenom_nom[nom_auteur]] = re.search(NomPrenom,df.iloc[i,nom_auteur]).group(2)
                df.iloc[i,nom_auteur]=re.sub(NomPrenom,r'\1',df.iloc[i,nom_auteur])
            
            

In [15]:
# for i in range (len(df)):
# #     print(df.iloc[i,0],type(df.iloc[i,2]))
#     if df.iloc[i,2] is not None and df.iloc[i,2] != "":
# #         print(df.iloc[i,0],df.iloc[i,2])
#         if re.search(NomPrenom,df.iloc[i,2]) :
#             df.iloc[i,10] = re.search(NomPrenom,df.iloc[i,2]).group(2)
#             df.iloc[i,2]=re.sub(NomPrenom,r'\1',df.iloc[i,2])

In [16]:
df

,Unnamed: 0,article_id,article_auteur,article_titre,article_page,revue_annee,revue_numero,article_auteur2,article_auteur3,article_auteur4,article_auteur1_prenom,article_auteur2_prenom,article_auteur_prenom,article_auteur4_prenom
0,0,BSPE-1-1,LAVERAN,Discours d’ouverture.,1,1908,1,,,,,,,
1,1,BSPE-1-15,MARTIN,Sur les symptômes nerveux du début de la malad...,15,1908,1,DARRÉ,,,,,,
2,2,BSPE-1-18,SERGENT,La fièvre méditerranéenne en Algérie. Note pré...,18,1908,1,,,,,,,
3,3,BSPE-1-22,REMY,Le Debab dans la région de Barika.,22,1908,1,,,,,,,
4,4,BSPE-1-25,SERGENT,Le Debab dans la région de la Zousfana (SudOra...,25,1908,1,LEDOUX,SERGENT,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2438,2438,BSPE-14-662,ROUBAUD,Le bien-être dans le Paludisme et les maladies...,662,1921,14,,,,,,,
2439,2439,BSPE-14-666,JOUVEAU-DUBREUIL,Le Béribéri au Setchouan,666,1921,14,,,,,,,
2440,2440,BSPE-14-685,DONATIEN,Le Paludisme en Corse. Etude épidémiologique.,685,1921,14,PARROT,SERGENT,SERGENT,,,,
2441,2441,BSPE-14-710,LAVIER,Flagellés parasites intestinaux du campagnol i...,710,1921,14,,,,,,,
